In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision.models.feature_extraction import create_feature_extractor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
def load_feature():
  img_feature = np.load("/content/drive/MyDrive/BiT/feature.npy")
  img_feature = torch.from_numpy(img_feature)
  return img_feature.to(device)

def load_label():
  img_label = np.load("/content/drive/MyDrive/BiT/label.npy")
  img_label = torch.from_numpy(img_label)
  return img_label

class tensordot(nn.Module):
  def __init__(self, img_feature):
    super().__init__()
    self.img_feature = img_feature

  def forward(self, num_vid):
    self.num_vid = torch.transpose(num_vid, 0, 1)
    return torch.tensordot(self.img_feature, self.num_vid, dims=1)

class video2vec(nn.Module):
    def __init__(self, num_vid, dim, img_feature, ipv_label):
      super().__init__()
      self.dim = dim
      self.num_vid = num_vid
      self.img_feature = img_feature
      self.ipv_label = ipv_label
      self.ipv_label = self.ipv_label.to(torch.int)

      self.emb = nn.Embedding(self.num_vid, self.dim)
      nn.init.constant_(self.emb.weight, 0.0)
      nn.init.calculate_gain("relu")
      self.dot = tensordot(self.img_feature)

    def forward(self, input):
      vector = self.emb(input)
      vector = torch.index_select(vector, 0, self.ipv_label)
      inn_prod = self.dot(vector)
      inn_prod = nn.functional.normalize(inn_prod, dim = 0)
      return inn_prod, vector

In [ ]:
feature = load_feature()

In [ ]:
ipv =  torch.tensor([5, 6, 8, 3, 4, 4, 6, 7, 2, 3, 3, 3, 7, 6, 9, 5, 3, 2, 6, 4, 8, 9, 6, 2,
        7])
#split images to groups

In [ ]:
num_img = feature.shape[0]
dim = feature.shape[1]
num_vid = ipv.shape[0]
ipv_label = torch.empty(torch.sum(ipv))
for i in range(num_vid):
  ipv_label[torch.sum(ipv[:i]):torch.sum(ipv[:i+1])] = torch.full((ipv[i], ), i)
ipv_label = ipv_label.to(device)
input = torch.arange(num_vid).to(device)
target = torch.arange(num_img).to(device)

In [ ]:
model = video2vec(num_vid, dim, feature, ipv_label).to(device)

In [ ]:
soft = nn.Softmax(dim = 0)

In [ ]:
torch.set_printoptions(edgeitems = 30)

In [ ]:
criterion1 = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

def ce_loss(pred, target):
    return criterion1(pred, target)

def train(input, target):
    model.train()
    optimizer.zero_grad()
    inn_prod, vector = model(input)
    loss = ce_loss(inn_prod, target)
    loss.backward()
    optimizer.step()
    return loss

epochs = 1000

for epoch in range(epochs):
    train_loss = 0
    loss = train(input, target)
    train_loss = loss.item()
    if epoch % 10 == 9 or epoch == epochs - 1:
        print('epoch: {}, loss: {:.3}'.format(
            epoch + 1,
            train_loss,
        ))

epoch: 10, loss: 4.6
epoch: 20, loss: 4.6
epoch: 30, loss: 4.6
epoch: 40, loss: 4.6
epoch: 50, loss: 4.6
epoch: 60, loss: 4.6
epoch: 70, loss: 4.6
epoch: 80, loss: 4.6
epoch: 90, loss: 4.6
epoch: 100, loss: 4.6
epoch: 110, loss: 4.6
epoch: 120, loss: 4.6
epoch: 130, loss: 4.6
epoch: 140, loss: 4.6
epoch: 150, loss: 4.6
epoch: 160, loss: 4.6
epoch: 170, loss: 4.6
epoch: 180, loss: 4.6
epoch: 190, loss: 4.6
epoch: 200, loss: 4.6
epoch: 210, loss: 4.6
epoch: 220, loss: 4.6
epoch: 230, loss: 4.6
epoch: 240, loss: 4.6
epoch: 250, loss: 4.6
epoch: 260, loss: 4.6
epoch: 270, loss: 4.6
epoch: 280, loss: 4.6
epoch: 290, loss: 4.6
epoch: 300, loss: 4.6
epoch: 310, loss: 4.6
epoch: 320, loss: 4.6
epoch: 330, loss: 4.6
epoch: 340, loss: 4.6
epoch: 350, loss: 4.6
epoch: 360, loss: 4.6
epoch: 370, loss: 4.6
epoch: 380, loss: 4.6
epoch: 390, loss: 4.6
epoch: 400, loss: 4.6
epoch: 410, loss: 4.6
epoch: 420, loss: 4.6
epoch: 430, loss: 4.6
epoch: 440, loss: 4.6
epoch: 450, loss: 4.6
epoch: 460, loss: 4

In [ ]:
extractor = create_feature_extractor(model, {"emb": "feature"})
weight = extractor(input)
print(weight["feature"].shape)
temp_weight = torch.unsqueeze(weight["feature"], 1)

torch.Size([25, 2048])


In [ ]:
lab = load_label()

In [ ]:
temp_feature = torch.unsqueeze(feature, 1)

In [ ]:
cos = nn.CosineSimilarity()

In [ ]:
for i in range(25):
  if(cos(temp_weight[0], temp_weight[i]) > 0.0):
    print(torch.sum(ipv[:i]))
    print(lab[torch.sum(ipv[:i]):torch.sum(ipv[:i+1])])
    print(cos(temp_weight[0], temp_weight[i]))
    print(" ")

tensor(0)
tensor([10, 83, 51, 26,  7])
tensor([1.], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(11)
tensor([79, 94, 54, 83, 88, 26, 70, 78])
tensor([0.1062], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(30)
tensor([33, 43, 49, 43, 57, 71])
tensor([0.0068], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(43)
tensor([23, 53])
tensor([0.0107], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(81)
tensor([36, 46, 55])
tensor([0.0058], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(84)
tensor([84, 48])
tensor([0.0516], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(86)
tensor([98, 37, 46,  1, 38, 90])
tensor([0.0231], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(104)
tensor([26,  0, 48, 63, 75,  3, 81, 10, 43])
tensor([0.0348], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(113)
tensor([68, 88, 29, 87, 31, 69])
tensor([0.1495], device='cuda:0', grad_fn=<SumBackward1>)
 
tensor(119)
tensor([68, 57])
tensor([0.2027], device='cuda:0', grad_fn=<SumBackward1>)
 
